# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [1]:
import sympy as sp
from ham_to_sparse_matrix import *
from constants import *
import scipy.sparse.linalg

In [2]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(4*aLat) + aLat*(qnP1-qnM1)*V(qn)/(4*aLat) )
fermionNI=sp.expand( (xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q_{n})**2/2 + q_{n+1}*V(q_{n})/4 - q_{n-1}*V(q_{n})/4 + V(q_{n})*q_{n+1}/4 - V(q_{n})*q_{n-1}/4 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [4]:
fermionNI + fermionI

-0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [5]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'dirichlet'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [6]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

a*V(q_{n})**2/2 + q_{n+1}*V(q_{n})/4 - q_{n-1}*V(q_{n})/4 + V(q_{n})*q_{n+1}/4 - V(q_{n})*q_{n-1}/4 - 0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [7]:
ham=0

lam=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
# ham.subs(boundaryConditions).doit() # this doesn't work?


def potential(n):
    return lam*qs[n]*qs[n]*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
ham

(4*\chi^{\dagger}_{0}*\chi_{1} + 4*\chi^{\dagger}_{1}*\chi_{0} + 4*\chi^{\dagger}_{1}*\chi_{2} + 4*\chi^{\dagger}_{2}*\chi_{1} + 4*p_{0}**2 + 4*p_{1}**2 + 4*p_{2}**2 - 2*q_{0}*q_{1}**3 - q_{0}*q_{2} + 13.0*q_{0}**2 - 24*q_{0}**2*\chi^{\dagger}_{0}*\chi_{0} + 2*q_{0}**3*q_{1} + 4*q_{0}**6 + 2*q_{1}*q_{0}**3 - 2*q_{1}*q_{2}**3 - 10.0*q_{1}**2 + 24*q_{1}**2*\chi^{\dagger}_{1}*\chi_{1} - 2*q_{1}**3*q_{0} + 2*q_{1}**3*q_{2} + 4*q_{1}**6 - q_{2}*q_{0} + 2*q_{2}*q_{1}**3 + 13.0*q_{2}**2 - 24*q_{2}**2*\chi^{\dagger}_{2}*\chi_{2} - 2*q_{2}**3*q_{1} + 4*q_{2}**6)/8

In [8]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

\chi^{\dagger}_{0}*\chi_{1}/2 + \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{2}/2 + \chi^{\dagger}_{2}*\chi_{1}/2 - a^{\dagger}_{0}*a^{\dagger}_{1}*a_{1}*a^{\dagger}_{1}/16 - a^{\dagger}_{0}*a^{\dagger}_{1}*a_{1}**2/16 - a^{\dagger}_{0}*a^{\dagger}_{1}**2*a_{1}/16 - a^{\dagger}_{0}*a^{\dagger}_{1}**3/16 - a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 17*a^{\dagger}_{0}*a_{0}/16 - 3*a^{\dagger}_{0}*a_{0}*\chi^{\dagger}_{0}*\chi_{0}/2 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a^{\dagger}_{1}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}**2*a^{\dagger}_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}**3/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{1}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2*a_{0}*a^{\dagger}_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2*a_{0}**2/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**3*a_{0}/16 + a^{\dagger}_{0}*

In [9]:
for cutoff in [2,3,4,5,6,7,8,9,10]:
    t=Timer('toHam')
    t.start()
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    t.stop()
    
    t=Timer('evals')
    t.start()
    ens=scipy.sparse.linalg.eigs(hamMat,k=10,sigma=0)[0]
    t.stop()
    
    print("Lambda = ",cutoff)
    print(ens)
    print()

toHam timer took: 4.8241 seconds
evals timer took: 0.0144 seconds
Lambda =  2
[0.15418747+2.0541016e-09j 0.15418752+9.4954211e-10j
 0.3124992 -8.1665847e-09j 0.15418744+1.4786428e-09j
 0.1541874 +6.6308936e-09j 0.3124998 -1.2140514e-08j
 0.31249988+2.4498162e-10j 0.3125    -3.5724772e-09j
 0.3125    +1.3171400e-09j 0.3124999 +1.4137790e-09j]

toHam timer took: 6.7308 seconds
evals timer took: 0.0040 seconds
Lambda =  3
[ 0.06825045-6.3725347e-10j -0.124735  -7.8500308e-09j
 -0.17239861+1.8576137e-09j  0.08979215+4.3345638e-09j
  0.09699035-2.6509290e-09j  0.10679831+2.4493747e-09j
 -0.2713635 +3.5877163e-09j  0.14073406-2.7249634e-09j
  0.17448147+2.6539222e-09j  0.28970635+6.6117772e-10j]

toHam timer took: 11.9639 seconds
evals timer took: 0.0198 seconds
Lambda =  4
[0.16912556+7.1226385e-09j 0.96722734+1.4004891e-08j
 1.0191715 +3.8445673e-09j 1.0281236 +1.3880791e-08j
 1.1001977 -4.1490690e-08j 1.2741709 -1.4274924e-08j
 1.2734082 +2.3284169e-08j 1.8046721 +1.9673767e-08j
 1.825033